# Tide Prediction Task

Start with some imports.

In [1]:
from gbdxtools import Interface
from shapely.wkt import loads
import dateutil.parser
from datetime import datetime

Create a GBDX interface.

In [2]:
gbdx = Interface()

Start with a catalog ID (this is some image of the Palm Islands in Dubai).

In [3]:
cat_id = '103001000349F800'

Get metadata record for parsing of footprint WKT and timestamp.

In [4]:
record = gbdx.catalog.get(cat_id)

Get the centroid of the footprint.

In [5]:
centroid = loads(record.get('properties').get('footprintWkt')).centroid

The latitude corresponds to `y` and longitude corresponds to `x`.

In [6]:
lat = centroid.y
lon = centroid.x
print('lat=%f, lon=%f' % (lat, lon))

lat=25.113527, lon=55.133696


Next, we get the timestamp of the image.

In [7]:
timestamp = dateutil.parser.parse(record.get('properties').get('timestamp'))

The timestamp is converted to the date time group format that the tide prediction algorithm expects (Y-m-d-H-M).

In [8]:
dtg = datetime.strftime(timestamp, '%Y-%m-%d-%H-%M')

In [9]:
print('dtg="%s"' % dtg)

dtg="2010-01-12-07-12"


At this point, we'd like to call the task with

In [10]:
print('lat="%s", lon="%s", dtg="%s"' % (str(lat), str(lon), dtg))

lat="25.1135269383", lon="55.1336961813", dtg="2010-01-12-07-12"


and receive an output like

```json
{
  "minimumTide24Hours":null,
  "maximumTide24Hours":null,
  "currentTide":null
}
```

We have currently implemented tide prediction using a modified version of bf-tideprediction (just without the flask wrapper for endpoints). The image `chambbj/hello-gbdx` v0.0.9 is available as GBDX task `hello-gbdx-chambbj`. It takes the aforementioned lat, lon, dtg all as strings.

In [11]:
aoptask = gbdx.Task('AOP_Strip_Processor',
                 data=gbdx.catalog.get_data_location(cat_id),
                 bands='MS',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=True,
                 ortho_epsg='UTM')

tide_task = gbdx.Task('ShorelineDetection',
                      lat = str(lat),
                      lon = str(lon),
                      dtg = dtg,
                      image = aoptask.outputs.data.value)

The tide prediction json file will be saved to `/mnt/work/output/json` and is persisted to `some_new_folder` within my user bucket/prefix on S3.

In [12]:
import uuid
from os.path import join
workflow = gbdx.Workflow([aoptask, tide_task])
random_str = str(uuid.uuid4())
workflow.savedata(tide_task.outputs.results, location=join('some_random_folder', random_str))
workflow.execute()
workflow.status

{u'event': u'submitted', u'state': u'pending'}

Check periodically for status `complete`.

In [14]:
import time
import sys
from __future__ import print_function
while not workflow.status.get('state') == u'complete':
    print(datetime.now(), workflow.status, end='\r')
    sys.stdout.flush()
    time.sleep(5.0)
workflow.status

{u'event': u'succeeded', u'state': u'complete'}

And download the result. Yes, we could display it here, but this is good enough.

In [342]:
gbdx.s3.download(join('some_random_folder', random_str))

The tides.json file is now in the local directory.

In [30]:
print(join('some_random_folder', random_str))

some_random_folder/bfd5995e-fe99-45f0-a078-fa65f0a79c7c


In [31]:
workflow.events

[{u'event': u'submitted',
  u'note': u'',
  u'state': u'pending',
  u'task': u'AOP_Strip_Processor_ac0d09ec',
  u'task_id': u'4624316097369987131',
  u'timestamp': u'2017-06-20T13:10:36.654311+00:00',
  u'when': u'15 minutes ago'},
 {u'event': u'submitted',
  u'note': u'',
  u'state': u'pending',
  u'task': u'hello-gbdx-chambbj_768d0621',
  u'task_id': u'4624316097374865595',
  u'timestamp': u'2017-06-20T13:10:36.658154+00:00',
  u'when': u'15 minutes ago'},
 {u'event': u'scheduled',
  u'note': u'instance_id: i-c92fa74a',
  u'state': u'pending',
  u'task': u'AOP_Strip_Processor_ac0d09ec',
  u'task_id': u'4624316097369987131',
  u'timestamp': u'2017-06-20T13:10:50.216415+00:00',
  u'when': u'15 minutes ago'},
 {u'event': u'started',
  u'note': u'instance_id: i-0cbf1f269de9d4abc, domain: raid',
  u'state': u'running',
  u'task': u'AOP_Strip_Processor_ac0d09ec',
  u'task_id': u'4624316097369987131',
  u'timestamp': u'2017-06-20T13:10:51.099878+00:00',
  u'when': u'15 minutes ago'},
 {u'ev

In [32]:
workflow.stdout

[{'id': u'4624316097374865595',
  'name': u'hello-gbdx-chambbj_768d0621',
  'stdout': u'<15>1 2017-06-20T13:24:19.864453Z 26f722e39492 beachfront 16 - (beachfront.logger:init_logger) DEBUG Initialized logger\n<14>1 2017-06-20T13:24:19.936946Z 26f722e39492 beachfront 16 - [bfaudit@48851 actor="bfalg_ndwi.ndwi" action="Open file" actee="/mnt/work/input/image/056215432010_01/056215432010_01_assembly.tif"] (bfalg_ndwi.ndwi:open_image) INFO Opening /mnt/work/input/image/056215432010_01/056215432010_01_assembly.tif\n<14>1 2017-06-20T13:24:19.939626Z 26f722e39492 beachfront 16 - (bfalg_ndwi.ndwi:main) INFO bfalg-ndwi start: bf\n<14>1 2017-06-20T13:24:19.939728Z 26f722e39492 beachfront 16 - [bfaudit@48851 actor="bfalg_ndwi.ndwi" action="Save file" actee="/mnt/work/output/results/bf_ndwi.tif"] (bfalg_ndwi.ndwi:process) INFO Saving NDWI to file /mnt/work/output/results/bf_ndwi.tif\n<14>1 2017-06-20T13:24:24.505119Z 26f722e39492 beachfront 16 - [bfaudit@48851 actor="beachfront.process" action="Re